In [14]:
import pandas as pd 

In [15]:
#Cargamos datafames
df_tiendas = pd.read_excel(r'C:\Users\ga8ri\vscode\ETL\ventas.bikes\ventas_bikes\bikeshops.xlsx') #bikeshop.id 
df_bikes = pd.read_excel(r'C:\Users\ga8ri\vscode\ETL\ventas.bikes\ventas_bikes\Bikes.xlsx') # bike.id
df_ordenes = pd.read_excel(r'C:\Users\ga8ri\vscode\ETL\ventas.bikes\ventas_bikes\orderlines.xlsx') #bikeshop.id = order.line , bike.id = product.id


***1.-Unir los archivos en una sola tabla***

In [ ]:
#realizamos un merge de las tablas por su clave primaria
df_combinado = pd.merge(df_ordenes, df_tiendas, left_on='order.line', right_on='bikeshop.id')
df_final = pd.merge(df_combinado, df_bikes, left_on='product.id', right_on='bike.id')
df_final


,order.id,order.line,order.date,customer.id,product.id,quantity,bikeshop.id,bikeshop.name,location,bike.id,model,description,price
0,1,1,2011-01-07,2,48,1,1,Pittsburgh Mountain Machines,"Pittsburgh, PA",48,Jekyll Carbon 2,Mountain - Over Mountain - Carbon,6070
1,1,2,2011-01-07,2,52,1,2,Ithaca Mountain Climbers,"Ithaca, NY",52,Trigger Carbon 2,Mountain - Over Mountain - Carbon,5970
2,2,1,2011-01-10,10,76,1,1,Pittsburgh Mountain Machines,"Pittsburgh, PA",76,Beast of the East 1,Mountain - Trail - Aluminum,2770
3,2,2,2011-01-10,10,52,1,2,Ithaca Mountain Climbers,"Ithaca, NY",52,Trigger Carbon 2,Mountain - Over Mountain - Carbon,5970
4,3,1,2011-01-10,6,2,1,1,Pittsburgh Mountain Machines,"Pittsburgh, PA",2,Supersix Evo Hi-Mod Team,Road - Elite Road - Carbon,10660
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15639,2000,4,2015-12-25,15,14,1,4,Detroit Cycles,"Detroit, MI",14,CAAD Disc Ultegra,Road - Elite Road - Aluminum,2660
15640,2000,5,2015-12-25,15,90,1,5,Cincinnati Speed,"Cincinnati, OH",90,Trail 2,Mountain - Sport - Aluminum,1350
15641,2000,6,2015-12-25,15,17,1,6,Louisville Race Equipment,"Louisville, KY",17,CAAD12 105,Road - Elite Road - Aluminum,1680
15642,2000,7,2015-12-25,15,70,1,7,Nashville Cruisers,"Nashville, TN",70,F-Si Carbon 4,Mountain - Cross Country Race - Carbon,2880


***2 Obtener el total de cantidad y precio de toda la venta de biciletas***

In [ ]:
df_final['Total'] = df_final['price']*df_final['quantity']

***3 Obtener el total de bicicletas tipo Montain y Road***

In [ ]:
#creamos columna con categoria otros para luego reemplazar 
df_final['Category'] ='Otros'
df_final.loc[df_final['description'].str.contains('Mountain', case=False), 'Category'] = 'Mountain'
df_final.loc[df_final['description'].str.contains('Road', case=False), 'Category'] = 'Road'


In [ ]:
total_mountain = df_final[df_final['Category'] == 'Mountain']['Total'].sum()
total_road = df_final[df_final['Category'] == 'Road']['Total'].sum()

In [ ]:
print(f'Total Mountain {total_mountain} ,Total Road {total_road}')

Total Mountain 39154735 ,Total Road 31877595


***4 Crear la columna 'states' a partir de la columna locacion solo para las ventas que contengan el modelo Supersix***

In [ ]:
df_final.columns

Index(['order.id', 'order.line', 'order.date', 'customer.id', 'product.id',
       'quantity', 'bikeshop.id', 'bikeshop.name', 'location', 'bike.id',
       'model', 'description', 'price', 'Total', 'Category'],
      dtype='object')

In [ ]:
#hacemos un df para supersix
df_supersix = df_final
#creo la columna Supersix para identificar productos con este modelo
df_supersix['Supersix'] = 'Others'
df_supersix['Supersix']=df_final['model'].str.contains('Supersix')


In [ ]:
#filtramos el df por supoersix y eliminamos todos los Valores falsos 
df_supersix = df_supersix.loc[df_supersix['Supersix']== True]
#eliminamos la columna Supersix para ahorrar espacio y no generar redundancia en los datos 
df_supersix = df_supersix.drop(columns=['Supersix'])

In [ ]:
#creamos la columna state, utilizando las dos ultimos caracteres de la columna location 
df_supersix['state'] =df_supersix['location']
#eliminamos espacios
df_supersix['state'].str.strip()
#extraemos los dos utimos caracteres de cada fila
df_supersix['state'] = df_supersix['location'].str[-2:]
df_supersix



    5 Crear una columna con los cuartiles ALTO , MEDIO y BAJO de ventas del modelo supersix que pertenescan al estado de Florida (FL) 

In [ ]:
df_supersix

,order.id,order.line,order.date,customer.id,product.id,quantity,bikeshop.id,bikeshop.name,location,bike.id,model,description,price,Total,Category,state
4,3,1,2011-01-10,6,2,1,1,Pittsburgh Mountain Machines,"Pittsburgh, PA",2,Supersix Evo Hi-Mod Team,Road - Elite Road - Carbon,10660,10660,Road,PA
6,3,3,2011-01-10,6,1,1,3,Columbus Race Equipment,"Columbus, OH",1,Supersix Evo Black Inc.,Road - Elite Road - Carbon,12790,12790,Road,OH
7,3,4,2011-01-10,6,4,1,4,Detroit Cycles,"Detroit, MI",4,Supersix Evo Hi-Mod Dura Ace 2,Road - Elite Road - Carbon,5330,5330,Road,MI
18,7,1,2011-01-12,9,2,1,1,Pittsburgh Mountain Machines,"Pittsburgh, PA",2,Supersix Evo Hi-Mod Team,Road - Elite Road - Carbon,10660,10660,Road,PA
20,7,3,2011-01-12,9,10,1,3,Columbus Race Equipment,"Columbus, OH",10,Supersix Evo Tiagra,Road - Elite Road - Carbon,1840,1840,Road,OH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15595,1994,6,2015-12-22,16,9,1,6,Louisville Race Equipment,"Louisville, KY",9,Supersix Evo 105,Road - Elite Road - Carbon,2240,2240,Road,KY
15599,1994,10,2015-12-22,16,7,2,10,Kansas City 29ers,"Kansas City, KS",7,Supersix Evo Ultegra 3,Road - Elite Road - Carbon,3200,6400,Road,KS
15622,1995,22,2015-12-23,10,10,1,22,Ann Arbor Speed,"Ann Arbor, MI",10,Supersix Evo Tiagra,Road - Elite Road - Carbon,1840,1840,Road,MI
15627,1995,27,2015-12-23,10,9,1,27,San Antonio Bike Shop,"San Antonio, TX",9,Supersix Evo 105,Road - Elite Road - Carbon,2240,2240,Road,TX


In [ ]:
# Filtrar las filas donde el estado es 'FL' y el modelo es 'Supersix'
df_fl_supersix = df_supersix[(df_supersix['state'] == 'FL')]

# Calcular los cuantiles
q_high = df_fl_supersix['Total'].quantile(0.75)
q_low = df_fl_supersix['Total'].quantile(0.25)

# Creamos rangos en las columnas segun lo solicitado
df_fl_supersix['cuartil_revenue'] = pd.cut(df_fl_supersix['Total'], bins=[-np.inf, q_low, q_high, np.inf])

# reemplazamos los valores por los de las etiquetas 
etiquetas = ['Low', 'Medium', 'High']
df_fl_supersix['cuartil_revenue'] = pd.cut(df_fl_supersix['Total'], bins=[-np.inf, q_low, q_high, np.inf], labels=etiquetas)

# Mostrar el DataFrame resultante
df_fl_supersix







NameError: name 'np' is not defined